# Preparação dos conjuntos de treino e teste para o TPZ

_Autores: Andreia Dourado, Bruno Moraes_

__Descrição: Divisão do training set nos conjuntos de treino(70%) e teste(30%), salvos em arquivos .hdf5 no formato compatível com o algortimo TPZ.__

### 1. Importando as bibliotecas

In [ ]:
import pandas as pd
import tables_io
import h5py
import numpy as np

### 2. Lendo o arquivo com os dados:

In [ ]:
data_convert=pd.read_csv('/home/andreia.dourado/TCC/dp02/object/10sigma/data/training_set_dp02_truth_obj_cut_limit_detect_10sigma.csv')

In [ ]:
data_convert.columns

In [ ]:
columns_map = {
    'coord_radp02_object': 'ra',
    'coord_decdp02_object': 'dec',
    'redshiftdp01_test_truth': "redshift",
    'mag_udp02_object': 'mag_u',
    'mag_gdp02_object': 'mag_g',
    'mag_rdp02_object': 'mag_r',
    'mag_idp02_object': 'mag_i',
    'mag_zdp02_object': 'mag_z',
    'mag_ydp02_object': 'mag_y',
    'magerr_udp02_object': 'magerr_u',
    'magerr_gdp02_object': 'magerr_g',
    'magerr_rdp02_object': 'magerr_r',
    'magerr_idp02_object': 'magerr_i',
    'magerr_zdp02_object': 'magerr_z',
    'magerr_ydp02_object': 'magerr_y'
}
data_convert = data_convert.rename(columns=columns_map)

In [ ]:
data_convert.columns

In [ ]:
data_convert = data_convert[['ra','dec','redshift','mag_u','mag_g','mag_r','mag_i','mag_z','mag_y','magerr_u', 'magerr_g',
       'magerr_r', 'magerr_i', 'magerr_z', 'magerr_y']]
data_convert

### 3. Arquivo de treino

#### 3.1 Selecionando a fração de objetos para o treino:

In [ ]:
fraction = int(0.7 * len(data_convert))
fraction

In [ ]:
training_csv = data_convert.sample(fraction,random_state=40)
training_csv

#### 3.2 Criando o aqrquivo .hdf5:

In [ ]:
train_file_path = '/home/andreia.dourado/TCC/dp02/object/10sigma/runs/detection_limits/train_file_dp02_object_10sigma_mag17.hdf5'

In [ ]:
train_dict_list = training_csv.to_dict(orient='list')

In [ ]:
with h5py.File(train_file_path, 'w') as train_file:
    #photometry_group = hdf5_file.create_group('photometry')
    def write_dict(group, data):
        for key, value in data.items():
            if isinstance(value, dict):
                # If the value is a dictionary, create a subgroup and recurse
                subgroup = group.create_group(str(key))
                write_dict(subgroup, value)
            else:
                # Otherwise, write the key-value pair
                group[str(key)] = value

    write_dict(train_file, train_dict_list)

#### 3.3 Verificando o arquivo:

In [ ]:
train_table = tables_io.read(train_file_path, fmt='hdf5')
train_table

In [ ]:
len(train_table['mag_g'])

### 4. Arquivo de teste

#### 4.1 Selecionando os objetos restantes:

In [ ]:
validation= data_convert.drop(training_csv.index)
validation

#### 4.1 Criando o arquivo .hdf5:

In [ ]:
test_file_path = '/home/andreia.dourado/TCC/dp02/object/10sigma/runs/detection_limits/test_file_dp02_object_10sigma_mag17.hdf5'

In [ ]:
test_dict_list = validation.to_dict(orient='list')

In [ ]:
with h5py.File(test_file_path, 'w') as test_file:
    def write_dict(group, data):
        for key, value in data.items():
            if isinstance(value, dict):
                subgroup = group.create_group(str(key))
                write_dict(subgroup, value)
            else:
                group[str(key)] = value

    write_dict(test_file, test_dict_list)

#### 4.2 Verificando o arquivo:

In [ ]:
test_table = tables_io.read(test_file_path, fmt='hdf5')
test_table

In [ ]:
len(test_table['mag_g'])